<h1>L2-Regularized Logistic Regression</h1>

In [1]:
library(mlr)
source('../utils.r')

set.seed(42)

folder_name = '../../raw_data' 
file_name   = 'german.csv'

Loading required package: ParamHelpers
Warning message:
“replacing previous import ‘BBmisc::isFALSE’ by ‘backports::isFALSE’ when loading ‘mlr’”

<h2>1. Dataprep</h2>

In [2]:
data = read.csv(file=sprintf('%s/%s',folder_name,file_name))

In [3]:
cat(sprintf('NRow: %d\nNCol: %d',nrow(data), ncol(data)))
head(data)

NRow: 1000
NCol: 22

X,V1,V2,V3,V4,V5,V6,V7,V8,V9,⋯,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21
1,A11,6,A34,A43,1169,A65,A75,4,A93,⋯,A121,67,A143,A152,2,A173,good,A192,A201,1
2,A12,48,A32,A43,5951,A61,A73,2,A92,⋯,A121,22,A143,A152,1,A173,good,A191,A201,2
3,A14,12,A34,A46,2096,A61,A74,2,A93,⋯,A121,49,A143,A152,1,A172,bad,A191,A201,1
4,A11,42,A32,A42,7882,A61,A74,2,A93,⋯,A122,45,A143,A153,1,A173,bad,A191,A201,1
5,A11,24,A33,A40,4870,A61,A73,3,A93,⋯,A124,53,A143,A153,2,A173,bad,A191,A201,2
6,A14,36,A32,A46,9055,A65,A73,2,A93,⋯,A124,35,A143,A153,1,A172,bad,A192,A201,1


<p style='color: red'> ATENTION: </p>Specifically in R, in classification problem, target must be set as Factor.

In [4]:
data$V21 = as.factor(data$V21)

MLR works only with features and target, this means that others columns must be dorped.

In [5]:
drops = c('X')
data  = data[ , !(names(data) %in% drops)]
cat(sprintf('NRow: %d\nNCol: %d',nrow(data), ncol(data)))

NRow: 1000
NCol: 21

Function
```R
createDummyFeatures(obj, target = character(0L), method = "1-of-n",  cols = NULL)
```

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/createDummyFeatures

In [6]:
data = createDummyFeatures(obj = data, target = 'V21')
cat(sprintf('NRow: %d\nNCol: %d',nrow(data), ncol(data)))

NRow: 1000
NCol: 63

<h2>2. Modeling</h2>

Function
```R
makeLearner(cl, id = cl, predict.type = "response", predict.threshold = NULL, 
            fix.factors.prediction = FALSE, ..., par.vals = list(), config = list())
```
Param.:

* cl: [character(1)] Class of learner. By convention, all classification learners start with “classif.”. A list of all integrated learners is available on the learners help page < https://mlr-org.github.io/mlr-tutorial/release/html/integrated_learners/ >.
* predict: [character(1)] “response” (= labels) or “prob” (= probabilities and labels by selecting the ones with maximal probability). Default is “response”.
* par.vals: [list] Optional list of named (hyper)parameters. The arguments in ... take precedence over values in this list. We strongly encourage you to use one or the other to pass (hyper)parameters to the learner but not both.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeLearner

In [7]:
cl = "classif.LiblineaRL2LogReg"

List all parameters that can be used in this classifier. The value must be set in par.vals parameter.

In [8]:
getParamSet(cl)

                 Type  len   Def      Constr Req Tunable Trafo
type         discrete    -     0         0,7   -    TRUE     -
cost          numeric    -     1    0 to Inf   -    TRUE     -
epsilon       numeric    -     -    0 to Inf   -    TRUE     -
bias          logical    -  TRUE           -   -    TRUE     -
wi      numericvector <NA>     - -Inf to Inf   -    TRUE     -
cross         integer    -     0    0 to Inf   -   FALSE     -
verbose       logical    - FALSE           -   -   FALSE     -

In [9]:
learner = makeLearner(cl = cl
                     , predict.type = "prob"
                     , par.vals = list()
                     )

Function
```R
makeClassifTask(id = deparse(substitute(data)), data, target, weights = NULL, blocking = NULL, 
                positive = NA_character_, fixup.data = "warn", check.data = TRUE)
```
Param.:

* data: [data.frame] A data frame containing the features and target variable(s).
* target: [character(1)] Name of the target variable.
* positive: [character(1)] Positive class for binary classification (otherwise ignored and set to NA). Default is the first factor level of the target attribute.
* fixup.data: [character(1)] Should some basic cleaning up of data be performed? Currently this means removing empty factor levels for the columns. Possible coices are: “no” = Don't do it. “warn” = Do it but warn about it. “quiet” = Do it but keep silent. Default is “warn”.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeLearner

In [10]:
task = makeClassifTask( data = data
                      , target = 'V21'
                      , positive = '2'
                      , fixup.data = 'no'
                      )

Function:
```R
makeResampleDesc(method, predict = "test", ..., stratify = FALSE, stratify.cols = NULL)
```
Param.:

* method: [character(1)] “CV” for cross-validation, “LOO” for leave-one-out, “RepCV” for repeated cross-validation, “Bootstrap” for out-of-bag bootstrap, “Subsample” for subsampling, “Holdout” for holdout.
* predict: What to predict during resampling: “train”, “test” or “both” sets. Default is “test”.
* ... : [any] Further parameters for strategies.
    * iters [integer(1)] Number of iterations, for “CV”, “Subsample” and “Boostrap”.
    * split [numeric(1)] Proportion of training cases for “Holdout” and “Subsample” between 0 and 1. Default is 2/3.
    * reps [integer(1)] Repeats for “RepCV”. Here iters = folds * reps. Default is 10.
    * folds [integer(1)] Folds in the repeated CV for RepCV. Here iters = folds * reps. Default is 10.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeResampleDesc

In [11]:
resample = makeResampleDesc( method = "CV"
                           , iters = 10
                           , predict = 'both'
                           , stratify = FALSE
                           )

List of performance measures:

Doc.: http://mlr-org.github.io/mlr-tutorial/release/html/measures/

In [12]:
measures = list(mmce #MMCE 
               ,acc  #acuracia
               ,f1   #f1
               ,ppv  #precision
               ,tpr  #recall
               ,auc  #AUC
               ,gini #Gini
               ,timetrain #tempo execucao
               )

Function:
```R
resample(learner, task, resampling, measures, weights = NULL, models = FALSE, extract, 
         keep.pred = TRUE, ..., show.info = getMlrOption("show.info"))
```
Param.:

* learner: [Learner] The learner.
* task: [Task] The task.
* resampling: [ResampleInstance] Resampling strategy.
* measures: [Measure | list of Measure] Performance measure(s) to evaluate. Default is mean misclassification error (mmce)
* weights: [numeric] Optional, non-negative case weight vector to be used during fitting. If given, must be of same length as observations in task and in corresponding order. Overwrites weights specified in the task. By default NULL which means no weights are used unless specified in the task.
* models: [logical(1)] Should all fitted models be returned? Default is FALSE.
* keep.pred: [logical(1)] Keep the prediction data in the pred slot of the result object. If you do many experiments (on larger data sets) these objects might unnecessarily increase object size / mem usage, if you do not really need them. In this case you can set this argument to FALSE. Default is TRUE.
* show.info: [logical(1)] Print verbose output on console? Default is set via configureMlr.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/resample

In [13]:
r = resample(learner = learner
            ,task = task 
            ,resampling = resample 
            ,measures = measures
            #---------------------#
            ,models = TRUE
            ,keep.pred = FALSE
            ,show.info = TRUE)

[Resample] cross-validation iter 1: mmce.test.mean=0.24,acc.test.mean=0.76,f1.test.mean=0.25,ppv.test.mean= 0.8,tpr.test.mean=0.148,auc.test.mean=0.62,gini.test.mean=0.241,timetrain.test.mean=0.006
[Resample] cross-validation iter 2: mmce.test.mean=0.34,acc.test.mean=0.66,f1.test.mean=   0,ppv.test.mean=   0,tpr.test.mean=   0,auc.test.mean=0.594,gini.test.mean=0.188,timetrain.test.mean=0.083
[Resample] cross-validation iter 3: mmce.test.mean=0.33,acc.test.mean=0.67,f1.test.mean=   0,ppv.test.mean=   0,tpr.test.mean=   0,auc.test.mean=0.567,gini.test.mean=0.133,timetrain.test.mean=0.004
[Resample] cross-validation iter 4: mmce.test.mean=0.32,acc.test.mean=0.68,f1.test.mean=   0,ppv.test.mean=   1,tpr.test.mean=   0,auc.test.mean=0.533,gini.test.mean=0.0662,timetrain.test.mean=0.006
[Resample] cross-validation iter 5: mmce.test.mean= 0.3,acc.test.mean= 0.7,f1.test.mean=   0,ppv.test.mean=   1,tpr.test.mean=   0,auc.test.mean=0.703,gini.test.mean=0.407,timetrain.test.mean=0.009
[Resample

<h2>3. Result Analysis</h2>

Train Measures

In [14]:
r$measures.train

iter,mmce,acc,f1,ppv,tpr,auc,gini,timetrain
1,0.2955556,0.7044444,0.10738255,0.6400000,0.05860806,0.5895975,0.1791951,0.006
2,0.2855556,0.7144444,0.11072664,0.7272727,0.05992509,0.6065167,0.2130335,0.083
3,0.2922222,0.7077778,0.03663004,0.8333333,0.01872659,0.5810214,0.1620427,0.004
4,0.2911111,0.7088889,0.06428571,0.7500000,0.03358209,0.5901663,0.1803325,0.006
5,0.2855556,0.7144444,0.14046823,0.7241379,0.07777778,0.6095297,0.2190594,0.009
6,0.2922222,0.7077778,0.10847458,0.6153846,0.05947955,0.6003688,0.2007376,0.008
7,0.2877778,0.7122222,0.08480565,0.7500000,0.04494382,0.6136109,0.2272219,0.004
8,0.2944444,0.7055556,0.09556314,0.7000000,0.05128205,0.6107168,0.2214335,0.012
9,0.2966667,0.7033333,0.08873720,0.7222222,0.04727273,0.5786182,0.1572364,0.008
10,0.2877778,0.7122222,0.12794613,0.7307692,0.07011070,0.5980148,0.1960295,0.005


Test Measures

In [15]:
r$measures.test

iter,mmce,acc,f1,ppv,tpr,auc,gini,timetrain
1,0.24,0.76,0.2500000,0.8000000,0.14814815,0.6204972,0.24099442,0.006
2,0.34,0.66,0.0000000,0.0000000,0.00000000,0.5938489,0.18769787,0.083
3,0.33,0.67,0.0000000,0.0000000,0.00000000,0.5667119,0.13342379,0.004
4,0.32,0.68,0.0000000,1.0000000,0.00000000,0.5330882,0.06617647,0.006
5,0.30,0.70,0.0000000,1.0000000,0.00000000,0.7033333,0.40666667,0.009
6,0.30,0.70,0.1176471,0.6666667,0.06451613,0.5694250,0.13884993,0.008
7,0.30,0.70,0.2500000,0.7142857,0.15151515,0.6160109,0.23202171,0.004
8,0.24,0.76,0.2000000,1.0000000,0.11111111,0.6463724,0.29274480,0.012
9,0.23,0.77,0.1481481,1.0000000,0.08000000,0.6448000,0.28960000,0.008
10,0.30,0.70,0.0000000,0.0000000,0.00000000,0.4949004,-0.01019913,0.005


Train Aggregated Result

In [16]:
apply(r$measures.train,2,mean)

iter       mmce        acc         f1        ppv        tpr        auc 
5.50000000 0.29088889 0.70911111 0.09650199 0.71931201 0.05217085 0.59781610 
      gini  timetrain 
0.19563221 0.01450000

Test Aggregated Result

In [17]:
apply(r$measures.test,2,mean)

iter       mmce        acc         f1        ppv        tpr        auc 
5.50000000 0.29000000 0.71000000 0.09657952 0.61809524 0.05552905 0.59889883 
      gini  timetrain 
0.19779765 0.01450000

Run Time in seconds

In [18]:
r$runtime

[1] 0.50544

<h2>4. Prediction for new data</h2>

Read the data to predict

In [19]:
new.data = read.csv(file=sprintf('%s/%s', folder_name, file_name))
new.data = createDummyFeatures(obj = new.data, target = 'V21')

Search for the best model in crossvalidation and use it to score the incoming data

In [20]:
best.model = which.max(r$measures.test$acc)

In [21]:
pred = predict(r$models[[best.model]], newdata = new.data)

Prediction Result

In [22]:
pred

Prediction: 1000 observations
predict.type: prob
threshold: 1=0.50,2=0.50
time: 0.00
  truth    prob.1       prob.2 response
1     1 0.8529108 1.470892e-01        1
2     2 1.0000000 1.148481e-09        1
3     1 0.6904802 3.095198e-01        1
4     1 0.9557165 4.428352e-02        1
5     2 0.9547498 4.525016e-02        1
6     1 0.7656937 2.343063e-01        1
... (1000 rows, 4 cols)


Cast result to data.frame to access the prediction

In [23]:
head(as.data.frame(pred))

truth,prob.1,prob.2,response
1,0.8529108,1.470892e-01,1
2,1.0000000,1.148481e-09,1
1,0.6904802,3.095198e-01,1
1,0.9557165,4.428352e-02,1
2,0.9547498,4.525016e-02,1
1,0.7656937,2.343063e-01,1
